In [1]:
import pandas as pd
from pandas.core.groupby.generic import DataFrameGroupBy
import numpy as np
from typing import Callable
from time import time
from termcolor import colored

color_text = lambda text: f"{'\033[1;92m'}{text}{'\033[0m'}"
extra_info = lambda extra='': f" {color_text("-->")} {extra}\n{'-' * 120}\n"

#####                                         Pandas Creation From Lists, Dict, Realworld Datasets
```js
        In Pandas DataFrame you can't set the datatype for each column. Its set datatype automatically. The dtype= is for only one datatype which will be applicable for all columns.. But in Polars DataFrame you can define the datatype manually.
```

#                                             Renaming Column and Index
```js
        1. We can do it by 'df.index = [...]' and 'df.columns = [...]' which changes PERMANENTLY.
        2. df.set_index(set_an_existing_column_an_index OR a new list or range(..))
        3. df.rename(you can rename SPECIFIC columns or index or both) and we need to pass 'dict' not 'ndarray or list'.
            i.  For 'columns' : df.rename( columns={old_column : new_column, old_column1 : new_column1.....} )
            ii. For 'index'   : df.rename( index=dict(zip(df.index, range(10, df.shape[0]+10))) )
        
        As we can see to Rename Index : df.set_index() is hassle free and for Columns : df.rename().
``` 

In [2]:
# from lists
info = [
    ['Maria0', 15, 16],   # every list is a ROW.
    ['Maria1', 18, 19],
    ['Maria2', 21, 22],
    ['Maria3', 24, 25]
]

pd.DataFrame(info, index=[10, 20, 30, 40], columns=['Name', 'Age', 'IQ'])

,Name,Age,IQ
10,Maria0,15,16
20,Maria1,18,19
30,Maria2,21,22
40,Maria3,24,25


In [3]:
# from dictionary
info = {
    'Name' : ['Maria0', 'Maria1', 'Maria2', 'Maria3'],
    'Age' : [15, 18, 21, 24],
    'IQ' : [16, 19, 22, 25]
}

pd.DataFrame(info, index=[10, 20, 30, 40]) # you can change the column name if you want.

,Name,Age,IQ
10,Maria0,15,16
20,Maria1,18,19
30,Maria2,21,22
40,Maria3,24,25


In [4]:
# from realworld datasets
df = pd.read_csv("D:\\datasets\\nba.csv")
df.tail(10)
# df.dtypes = ['object', 'object', 'int64', 'object', 'int64', 'object', 'int64', 'object', 'int64'] will give error because you can't directly change the dtypes like that, you need to cast each column.

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
448,Gordon Hayward,Utah Jazz,20.0,SF,26.0,6-8,226.0,Butler,15409570.0
449,Rodney Hood,Utah Jazz,5.0,SG,23.0,6-8,206.0,Duke,1348440.0
450,Joe Ingles,Utah Jazz,2.0,SF,28.0,6-8,226.0,NaN,2050000.0
451,Chris Johnson,Utah Jazz,23.0,SF,26.0,6-6,206.0,Dayton,981348.0
452,Trey Lyles,Utah Jazz,41.0,PF,20.0,6-10,234.0,Kentucky,2239800.0
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,NaN,900000.0
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,NaN,2900000.0
456,Jeff Withey,Utah Jazz,24.0,C,26.0,7-0,231.0,Kansas,947276.0
457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#                                                       DataFrame Attributes

In [5]:
print(df.shape, extra_info("shape"))
print(df.size, extra_info("size (total Values including Null Values)"))
print(df.columns, extra_info("columns name"))
print(df.dtypes, extra_info("dtypes"))

print(df.index, extra_info("index"))
df.set_index([np.arange(2, 460)], inplace=True)  # one way to set the index, set an Existing Column as index or a new list.
print(df.index, extra_info("index"))
df.index = np.arange(1, 459)                     # second way to set the index.
print(df.index, extra_info("index"))

print(df.values, extra_info("values(shown in 2D Numpy Array where each 1D Numpy Array represents a Row of the DataFrame)"))

(458, 9)  --> shape
------------------------------------------------------------------------------------------------------------------------

4122  --> size (total Values including Null Values)
------------------------------------------------------------------------------------------------------------------------

Index(['Name', 'Team', 'Number', 'Position', 'Age', 'Height', 'Weight',
       'College', 'Salary'],
      dtype='object')  --> columns name
------------------------------------------------------------------------------------------------------------------------

Name         object
Team         object
Number      float64
Position     object
Age         float64
Height       object
Weight      float64
College      object
Salary      float64
dtype: object  --> dtypes
------------------------------------------------------------------------------------------------------------------------

RangeIndex(start=0, stop=458, step=1)  --> index
--------------------------------------------

#                                                       DataFrame Methods

In [6]:
# head, tail, sample are same as series.
print(df.info(), extra_info("info(Index, Columns, Count of Non Null Values in each column, dtype)"))
print(df.describe(), extra_info("describe(Works only on Numerical Columns)"))

<class 'pandas.core.frame.DataFrame'>
Index: 458 entries, 1 to 458
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      457 non-null    object 
 1   Team      457 non-null    object 
 2   Number    457 non-null    float64
 3   Position  457 non-null    object 
 4   Age       457 non-null    float64
 5   Height    457 non-null    object 
 6   Weight    457 non-null    float64
 7   College   373 non-null    object 
 8   Salary    446 non-null    float64
dtypes: float64(4), object(5)
memory usage: 35.8+ KB
None  --> info(Index, Columns, Count of Non Null Values in each column, dtype)
------------------------------------------------------------------------------------------------------------------------

           Number         Age      Weight        Salary
count  457.000000  457.000000  457.000000  4.460000e+02
mean    17.678337   26.938731  221.522976  4.842684e+06
std     15.966090    4.404016   26.368343  5.22923

In [7]:
print(df.isna().sum(), extra_info("isna().sum() on each column")) # for horizontally, set axis=1 inside sum(..)
print(df.duplicated(), extra_info("duplicated(which row is Duplicated i.e. True and which are not i.e. False)"))

# df.columns = map(str, range(1, 10)) # one way to rename the columns
# print(df.head(1), extra_info("Column name changed"))
print(df.rename(columns={'Number':'Marks', 'Salary':'Wage'}), extra_info("renamed some columns")) # returns a new DF after renaming the columns we specified.
#               You can set "inplace=True" for permanent changes.

# We also df .sum()/.mean() etc and by default they are computed by vertically i.e. on Each Column. You can set axis=1 for doing it HORIZONTALLY.

Name         1
Team         1
Number       1
Position     1
Age          1
Height       1
Weight       1
College     85
Salary      12
dtype: int64  --> isna().sum() on each column
------------------------------------------------------------------------------------------------------------------------

1      False
2      False
3      False
4      False
5      False
       ...  
454    False
455    False
456    False
457    False
458    False
Length: 458, dtype: bool  --> duplicated(which row is Duplicated i.e. True and which are not i.e. False)
------------------------------------------------------------------------------------------------------------------------

              Name            Team  Marks Position   Age Height  Weight  \
1    Avery Bradley  Boston Celtics    0.0       PG  25.0    6-2   180.0   
2      Jae Crowder  Boston Celtics   99.0       SF  25.0    6-6   235.0   
3     John Holland  Boston Celtics   30.0       SG  27.0    6-5   205.0   
4      R.J. Hunter  Boston 

#                                               unique(), nunique(), hasnans()

In [8]:
uniq = df['Position'].unique() # works only on A Series i.e. on A Single Column.
print(uniq, type(uniq), uniq.size, extra_info("uniq, type(uniq), uniq.size")) # size also counts NaN.
print(df['Position'].nunique(), extra_info("Unique Non Null values.")) # drop_na = True by default, reason why NaN is not counted

# df.notna(), df.isna() works on Each Column.
print(df['Position'].hasnans, extra_info("hasnans on 'Position' Column")) # works on Series only
print(df.apply(lambda col: col.hasnans), extra_info("hasnans on Each Column"))

['PG' 'SF' 'SG' 'PF' 'C' nan] <class 'numpy.ndarray'> 6  --> uniq, type(uniq), uniq.size
------------------------------------------------------------------------------------------------------------------------

5  --> Unique Non Null values.
------------------------------------------------------------------------------------------------------------------------

True  --> hasnans on 'Position' Column
------------------------------------------------------------------------------------------------------------------------

Name        True
Team        True
Number      True
Position    True
Age         True
Height      True
Weight      True
College     True
Salary      True
dtype: bool  --> hasnans on Each Column
------------------------------------------------------------------------------------------------------------------------



#                                      dropna(axis=0, how='any', subset=None, inplace=False) by default

```js
what is 'axis'?
---------------
        axis =  0 means VERTICALLY / 'ROW WISE'. So we will move VERTICALLY / 'ROW WISE' and the moment we find a NAN, we will delete that 'ROW'.
                So if you want to 'DELETE' A 'ROW', you MOVE 'ROW WISE' / VERTICALLY.
        
        axis =  1 means HORIZONTALLY / 'COLUMN WISE'. So we will move HORIZONTALLY / 'COLUMN WISE' and the moment we find a NAN, we will delete that 'COLUMN'.
                So if you want to 'DELETE' A 'COLUMN', you MOVE 'COLUMN WISE' / HORIZONTALLY.
        
        `Always ask yourself "what I want to DELETE? ROW(vertically) or COLUMN(horizontally)?"`

What is 'how'?
--------------
                 name          toy             born
                ------      ---------       ----------
                'Alfred     None            NaT'        ---> This is HORIZONTAL MOVEMENT. how='any' means if AT LEAST ONE NaN found,
                Batman      Batmobile       1940-04-25                                    we delete the Entire COLUMN.
                Catwoman    Bullwhip        NaT                                           E.g. => toy[0] = None. So delete 'toy'.
                                                                                        
                                                                                          how = 'all' means if ENTIRE ROW is NaN, only
                                                                                          then we will delete that Column.

What is 'subset'?
-----------------
                If we want to do dropna(..) based on only SPECIFIC COLUMN / COLUMNS.
                        dropna(subset='name')    OR    dropna(subset=['name', 'marks'])
```

In [9]:
df_drop = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                        "toy" : [None, 'Batmobile', 'Bullwhip'],
                        "born": [None, pd.Timestamp("1940-04-25"), None]})

df_drop

,name,toy,born
0,Alfred,None,NaT
1,Batman,Batmobile,1940-04-25
2,Catwoman,Bullwhip,NaT


In [10]:
print(df_drop, extra_info("Main DataFrame"))

print(df_drop.dropna(axis=0, how='any').to_string(), extra_info("dropna(axis=0, how='any')"))
print(df_drop.dropna(axis=1, how='any').to_string(), extra_info("dropna(axis=1, how='any')"))

# delete ROW which is completely NULL i.e. rows with no single information.
print(df_drop.dropna(axis=0, how='all').to_string(), extra_info("dropna(axis=0, how='all')"))

# delete ROWS whose 'name' OR 'born' is NULL.
print(df_drop.dropna(subset=['name', 'born'], axis=0, how='any').to_string(), extra_info("dropna(subset=['name', 'born'], axis=0, how='any')"))
print(df_drop.dropna(subset=['toy', 'born'],  axis=0, how='all').to_string(), extra_info("dropna(subset=['toy', 'born'],  axis=0, how='all')"))

       name        toy       born
0    Alfred       None        NaT
1    Batman  Batmobile 1940-04-25
2  Catwoman   Bullwhip        NaT  --> Main DataFrame
------------------------------------------------------------------------------------------------------------------------

     name        toy       born
1  Batman  Batmobile 1940-04-25  --> dropna(axis=0, how='any')
------------------------------------------------------------------------------------------------------------------------

       name
0    Alfred
1    Batman
2  Catwoman  --> dropna(axis=1, how='any')
------------------------------------------------------------------------------------------------------------------------

       name        toy       born
0    Alfred       None        NaT
1    Batman  Batmobile 1940-04-25
2  Catwoman   Bullwhip        NaT  --> dropna(axis=0, how='all')
------------------------------------------------------------------------------------------------------------------------

     name      

#                                               df.drop_duplicates(..) only rows.
```jS
        drop_duplicates(subset=None, keep='first') by default. And by default it delete the duplicate ROWS. It doesnt work on Delete Duplicate 'columns', only on 'rows'.

        keep='first' : Among [1, 1, 3, 2, 1] it keeps the 'first' 1 and delete its next duplicate 1s.
            ='last'  : .................................. 'last'  1 .............. previous duplicate 1s.
        
        subset= A Single Column Name OR LIST of columns names based on what we want to delete duplicate rows :
                Lets say among ['name', 'age', 'marks', 'city'] I want to delete those rows whose 'name' and 'marks' are duplicated, doesnt matter if their 'age' and 'city' are also same or not. So drop_duplicates(subset=['name', 'marks'])
```

In [11]:
df1 = pd.read_csv("D:\\datasets\\drop_duplicate.csv")
print(df1, extra_info("df1 DataFrame"))
print(df1.drop_duplicates(), extra_info("drop_duplicates(Any Duplicate Row)"))
print(df1.drop_duplicates(keep='last'), extra_info("drop_duplicates(keep='last duplicate row, delete its prev duplicate rows')"))

# Delete those duplicate rows whose 'name' and 'marks' are same, 'city' and 'age' column can be different egal.
# keep the last duplicate row and delete its previous duplicate rows.
print(df1.drop_duplicates(subset=['name', 'marks'], keep='last'), extra_info("drop_duplicates(subset=['name', 'marks'], keep='last')"))

    name  marks    city  age
0  Maria     79   Dhaka   23
1  Maria     79  Khulna   25
2  Akira     89   Dhaka   21
3  Maria     79   Dhaka   23
4  Saria     76     Ctg   27  --> df1 DataFrame
------------------------------------------------------------------------------------------------------------------------

    name  marks    city  age
0  Maria     79   Dhaka   23
1  Maria     79  Khulna   25
2  Akira     89   Dhaka   21
4  Saria     76     Ctg   27  --> drop_duplicates(Any Duplicate Row)
------------------------------------------------------------------------------------------------------------------------

    name  marks    city  age
1  Maria     79  Khulna   25
2  Akira     89   Dhaka   21
3  Maria     79   Dhaka   23
4  Saria     76     Ctg   27  --> drop_duplicates(keep='last duplicate row, delete its prev duplicate rows')
------------------------------------------------------------------------------------------------------------------------

    name  marks   city  age
2  

#                                           df.drop(..) any (custom) rows or columns. Boolean mask not acceptable.

In [12]:
df2 = pd.read_csv("D:\\datasets\\drop_duplicate.csv")
print(df2, extra_info("df2 DataFrame"))

print(df2.drop(index=[0, 3]), extra_info("drop(index=[0, 3]")) # index = custom index
# df2.drop(index=(df2['name'] == 'Maria')) # boolean mask is not acceptable
print(df2.drop(columns=['city', 'marks']), extra_info("drop(columns=['city', 'marks'])"))

    name  marks    city  age
0  Maria     79   Dhaka   23
1  Maria     79  Khulna   25
2  Akira     89   Dhaka   21
3  Maria     79   Dhaka   23
4  Saria     76     Ctg   27  --> df2 DataFrame
------------------------------------------------------------------------------------------------------------------------

    name  marks    city  age
1  Maria     79  Khulna   25
2  Akira     89   Dhaka   21
4  Saria     76     Ctg   27  --> drop(index=[0, 3]
------------------------------------------------------------------------------------------------------------------------

    name  age
0  Maria   23
1  Maria   25
2  Akira   21
3  Maria   23
4  Saria   27  --> drop(columns=['city', 'marks'])
------------------------------------------------------------------------------------------------------------------------



#                                       df.apply(..) works on both `rows` and `columns`

```js
        apply on Series(a Single Column/Row) :
        --------------------------------------
            Series.apply(func= lambda v: ...) => apply() goes inside each value, v, and do the calculation written inside 'function'.
        
        apply on DataFrame() : by default axis=0 i.e. works on Each 'Column'
        --------------------------------------------------------------------
            DataFrame.apply(func= lambda 'column': ...) => apply() only GRABS EACH COLUMN and 'doesnt go inside that column' e.g. apply() grabs the column 'point1' but it wont go inside that column. We have to write again another function to go inside the column 'point1' and do our calculation on EACH VALUE INSIDE 'point1'.
                            
                        DataFrame.apply(func= lambda 'column': 'column'.apply(....))
        
        apply on DataFrame(axis=1) : works on Each 'Row'
        ------------------------------------------------
            DataFrame.apply(func= lambda 'row': 'row'.apply(....)) => Same as Above, except here apply(axis=1) grab each 'row' only and to go inside that 'row' we need another function.
    
    DataFrame.map(lambda v: ) works COLUMN by COLUMN and also goes inside that column. It only works for column, not for axis=1. BUT its slow compared to apply.
```

In [13]:
df3 = pd.DataFrame({'point1' : [1, 2, 3, 4, 5], 'point2' : [6, 7, 8, 9, 10]}, dtype='int8')
print(df3, '     ', extra_info("DataFrame df3"))

print(df3['point1'].apply(func=lambda v: v+1 if v&1 else v+2), extra_info("df3['point1'].apply() on a Series(A single Column)"))
print(df3.iloc[0].apply(func=lambda v: v+1 if v&1 else v+2), extra_info("df3.iloc[0].apply() on a Series(A single Row)"))

#                                                  apply() on DataFrame
return_columns_name: Callable[[pd.Series], str] = lambda col: col.name
print(df3.apply(return_columns_name).to_numpy(), extra_info("Columns Name using apply(..), axis=0 by default"))
# Ans for above = ['point1' 'point2']. So apply(by default axis=0) only grabs each Column but it doen't go inside that column.

calculate_inside_each_column: Callable[[pd.Series], pd.Series] = lambda column: column.apply(lambda v: v+1 if v&1 else v+2)
print(df3.apply(calculate_inside_each_column, axis=0), extra_info("df3.apply(calculate_inside_each_column, axis=0)"))
# Above : First we Grabbed a Column, then did our calculation Going Inside that Column. Same for every other column.

calculate_inside_each_row: Callable[[pd.Series], pd.Series] = lambda row: abs(row.iloc[0] - row.iloc[1]) ** 4
print(df3.apply(calculate_inside_each_row, axis=1), extra_info("df3.apply(calculate_inside_each_row, axis=1)"))
# Above : First we Grabbed a ROW i.e. A SERIES, then did our calculation on row[0] and row[1]. Same for every other row.

# Above : if df3.datatype was uint8. We wouldn't be able to do (Smaller uint8 - Larger uint8), it would cause OVERFLOW because
# (Smaller uint8 - Larger uint8) = A Negative Number and between 2 objects of uint, we can't produce any NEGATIVE NUMBER.
# (Larger uint8 - Smaller uint8) = uint8, OK. (Larger uint8 + Smaller uint8) = uint16 is also okay BECAUSE the DataTypes are 'uint' and 
# the output also a 'uint' BUT (Smaller uint8 - Larger uint8) = INT8 is NOT OKAY because the DataTypes are 'uint' but output is int.

# BUT if df3.datatype was uint8. Then we could do df3.astype('int8').apply(...)

   point1  point2
0       1       6
1       2       7
2       3       8
3       4       9
4       5      10        --> DataFrame df3
------------------------------------------------------------------------------------------------------------------------

0    2
1    4
2    4
3    6
4    6
Name: point1, dtype: int64  --> df3['point1'].apply() on a Series(A single Column)
------------------------------------------------------------------------------------------------------------------------

point1    2
point2    8
Name: 0, dtype: int64  --> df3.iloc[0].apply() on a Series(A single Row)
------------------------------------------------------------------------------------------------------------------------

['point1' 'point2']  --> Columns Name using apply(..), axis=0 by default
------------------------------------------------------------------------------------------------------------------------

   point1  point2
0       2       8
1       4       8
2       4      10
3       6      10
4

#                                                   df[column_names] [mask(optional)]
```js
        'column_names' can be A Single Column('Name') or A List Of Column Names(['Name', 'Age', 'Weight'])

        'mask' :

            Custom Rows i.e. 'Indexes'                           Boolen Mask(Series/list/ndArray)
            --------------------------                           --------------------------------
                    'Aria'                 -----------------                   True
                    'Maria'                                                    False
                    'Akira'                -----------------                   True
                    'Saria'                                                    False

        'Indexes' of a Series/DataFrame.                         But Only 'True' rows are will be displayed WITH all the column
        Assuming 'column_names' = all or                         specified in 'column_names'.
        specific columns selected.
        
        Why we cant select 'rows' here with slicing like df[:5, column_names][mask]? :
                So you want to select first 5 rows(:5) or (2:50:3) rows WITH specific columns? Yes, possible => 'iloc'
                        But also with 'mask' which length is a WHOLE COLUMN? That doesn't make much sense or practical implication.
                For this we have 'loc[]' and 'iloc'. Basically 'WE WONT USE THIS 'df[column_names][mask]' but THE loc and iloc method mostly'.

```

#                                      WARNING: DON'T USE THE ABOVE METHOD! USE `loc` and `iloc`.

In [14]:
print(df.head(5).to_string(), extra_info("df.head(5)"))

print(df['Name'], extra_info("df['Name']"))
print(df[['Name', 'ct marks', 'Team']], extra_info("df[['Name', 'ct marks', 'Team']]"))

mask2 = [True] * 229 + [False] * 229
column_names = ['Name', 'Weight']

print(df[column_names] [mask2], extra_info("df[column_names] [mask2]"))
print(df.loc[mask2, column_names], extra_info("df.loc[mask2, column_names]")) # loc explained below section.

            Name            Team  Number Position   Age Height  Weight            College     Salary
1  Avery Bradley  Boston Celtics     0.0       PG  25.0    6-2   180.0              Texas  7730337.0
2    Jae Crowder  Boston Celtics    99.0       SF  25.0    6-6   235.0          Marquette  6796117.0
3   John Holland  Boston Celtics    30.0       SG  27.0    6-5   205.0  Boston University        NaN
4    R.J. Hunter  Boston Celtics    28.0       SG  22.0    6-5   185.0      Georgia State  1148640.0
5  Jonas Jerebko  Boston Celtics     8.0       PF  29.0   6-10   231.0                NaN  5000000.0  --> df.head(5)
------------------------------------------------------------------------------------------------------------------------

1      Avery Bradley
2        Jae Crowder
3       John Holland
4        R.J. Hunter
5      Jonas Jerebko
           ...      
454     Shelvin Mack
455        Raul Neto
456     Tibor Pleiss
457      Jeff Withey
458              NaN
Name: Name, Length: 458, 

KeyError: "['ct marks'] not in index"

#                                       df.loc[..] and df.iloc[..]

#####                                    df.loc[custom_rows  OR `boolean mask(series/list/ndArray)`, columns_name]
#####                                   df.iloc[0 based rows OR `boolean mask(list/ndArray)`       , 0 based columns_indexes]

```js
What is 0 based indexes : What Pandas INTERNALLY sees the Custom Indexes we provide. Even if the index is like [0, 1, 2, 3...], it's the Custom Index we provided or Pandas Generated. Pandas sees this Custome Index ([0, 1, 2, 3..]) as [0, 1, 2, 3..] INTERNALLY which is same as the Custom Index in this case.

How boolean mask also works here?

            Custom Rows i.e. 'Indexes'                           Boolen Mask(Series/list/ndArray)
            --------------------------                           --------------------------------
                    'Aria'                 -----------------                   True
                    'Maria'                                                    False
                    'Akira'                -----------------                   True
                    'Saria'                                                    False

        'Indexes' of a Series/DataFrame.                         But Only 'True' rows are will be displayed.
        Assuming all or specific columns
        selected which doesn't matter.
```

In [ ]:
df.head(5)

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
1,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
2,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
3,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
4,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
5,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [ ]:
print(df.iloc[:4, [0, 4, 6]], extra_info("First 4 rows for 1st, 5th and 7th Column")) # 0 based Indexing, Pandas Internal Indexing.
print(df.loc[:4, ['Name', 'Age', 'Weight']], extra_info("First 4 rows for the columns Name, Age and Weight")) # Our custom indexs and column names.

mask = (df['Age'] > 30) & (df['Position'] == 'PG')
print(df.loc[mask, ['Name', 'Height']], extra_info("df.loc[mask, ['Name', 'Height']]"))

print(df.iloc[np.array(mask), [0, 5]], extra_info("df.iloc[np.array(mask), [0, 5]]")) # Series boolean mask not acceptable. So np.array(mask) or list(mask).

            Name  Age  Weight
1  Avery Bradley   25   180.0
2    Jae Crowder   25   235.0
3   John Holland   27   205.0
4    R.J. Hunter   22   185.0  --> First 4 rows for 1st, 5th and 7th Column
------------------------------------------------------------------------------------------------------------------------

            Name  Age  Weight
1  Avery Bradley   25   180.0
2    Jae Crowder   25   235.0
3   John Holland   27   205.0
4    R.J. Hunter   22   185.0  --> First 4 rows for the columns Name, Age and Weight
------------------------------------------------------------------------------------------------------------------------

                Name Height
20      Jarrett Jack    6-3
35     Jose Calderon    6-3
101       Chris Paul    6-0
103   Pablo Prigioni    6-3
113  Marcelo Huertas    6-3
132     Ronnie Price    6-2
153     Aaron Brooks    6-0
181      Mo Williams    6-1
184      Steve Blake    6-3
229       J.J. Barea    6-0
231   Raymond Felton    6-1
232     Devin Harri

In [ ]:
df.sample(5)

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
392,Joffrey Lauvergne,Denver Nuggets,77.0,C,24.0,6-11,220.0,NaN,1709719.0
407,Tayshaun Prince,Minnesota Timberwolves,12.0,SF,36.0,6-9,212.0,Kentucky,947276.0
227,Rashad Vaughn,Milwaukee Bucks,20.0,SG,19.0,6-6,202.0,UNLV,1733040.0
55,T.J. McConnell,Philadelphia 76ers,12.0,PG,24.0,6-2,200.0,Arizona,525093.0
235,JaVale McGee,Dallas Mavericks,11.0,C,28.0,7-0,270.0,Nevada,1270964.0


In [ ]:
#                                       A Q : Find the Team whose Name has the word 'Avery'.
# you can't write 'Avery' in df['Name']  because it will just check if 'Avery' value inside df['Name'] column, False.
mask0 = df['Name'].apply(lambda name: pd.notna(name) and 'Avery' in name) # custom logic, slower as lambda a python function.
mask1 = df['Name'].str.contains('Avery', na=False) # df['Name'].str returns a StringMethod Object telling us that we can now apply string methods on df['Name'] column.
# na = False means Nan value won't be checked and will return False if Nan value found.
print(df.loc[mask0, 'Team'], extra_info("Name of The Team whose Name has the word 'Avery'"))
print(df.loc[mask1, 'Team'], extra_info("Name of The Team whose Name has the word 'Avery'")) # faster because df[..].str.contains(..) written in C.
print(df.loc[mask0, 'Team']._is_view, df.loc[mask1, 'Team']._is_view) # False i.e. They are New Series.

1    Boston Celtics
Name: Team, dtype: object  --> Name of The Team whose Name has the word 'Avery'
------------------------------------------------------------------------------------------------------------------------

1    Boston Celtics
Name: Team, dtype: object  --> Name of The Team whose Name has the word 'Avery'
------------------------------------------------------------------------------------------------------------------------

False False


#                                                   Adding a New Column

In [ ]:
df['Country'] = 'Bangladesh 2.0'
print(df.head(), extra_info("A Column name Country added where each value is Bangladesh"))

# df['ct marks'] = [5, 9, 10, 2, 6] # ValueError: Length of values (5) does not match length of index (458)
df['ct marks'] = [5, 9, 10, 2, 6] + [np.nan] * 453 # if you pass list, you've to pass ALL VALUES.
print(df.head(10))

            Name            Team  Number Position   Age Height  Weight  \
1  Avery Bradley  Boston Celtics     0.0       PG  25.0    6-2   180.0   
2    Jae Crowder  Boston Celtics    99.0       SF  25.0    6-6   235.0   
3   John Holland  Boston Celtics    30.0       SG  27.0    6-5   205.0   
4    R.J. Hunter  Boston Celtics    28.0       SG  22.0    6-5   185.0   
5  Jonas Jerebko  Boston Celtics     8.0       PF  29.0   6-10   231.0   

             College     Salary         Country  
1              Texas  7730337.0  Bangladesh 2.0  
2          Marquette  6796117.0  Bangladesh 2.0  
3  Boston University        NaN  Bangladesh 2.0  
4      Georgia State  1148640.0  Bangladesh 2.0  
5                NaN  5000000.0  Bangladesh 2.0    --> A Column name Country added where each value is Bangladesh
------------------------------------------------------------------------------------------------------------------------

             Name            Team  Number Position   Age Height  Weig

#                                                   astype()

In [ ]:
df['Position'] = df['Position'].astype('category')
df['Age'] = df['Age'].fillna(0).astype('uint8')
df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Country,ct marks
1,Avery Bradley,Boston Celtics,0.0,PG,25,6-2,180.0,Texas,7730337.0,Bangladesh 2.0,5.0
2,Jae Crowder,Boston Celtics,99.0,SF,25,6-6,235.0,Marquette,6796117.0,Bangladesh 2.0,9.0
3,John Holland,Boston Celtics,30.0,SG,27,6-5,205.0,Boston University,NaN,Bangladesh 2.0,10.0
4,R.J. Hunter,Boston Celtics,28.0,SG,22,6-5,185.0,Georgia State,1148640.0,Bangladesh 2.0,2.0
5,Jonas Jerebko,Boston Celtics,8.0,PF,29,6-10,231.0,NaN,5000000.0,Bangladesh 2.0,6.0


#                                   value_counts() and sort_values()

In [ ]:
df

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Country,ct marks
1,Avery Bradley,Boston Celtics,0.0,PG,25,6-2,180.0,Texas,7730337.0,Bangladesh 2.0,5.0
2,Jae Crowder,Boston Celtics,99.0,SF,25,6-6,235.0,Marquette,6796117.0,Bangladesh 2.0,9.0
3,John Holland,Boston Celtics,30.0,SG,27,6-5,205.0,Boston University,NaN,Bangladesh 2.0,10.0
4,R.J. Hunter,Boston Celtics,28.0,SG,22,6-5,185.0,Georgia State,1148640.0,Bangladesh 2.0,2.0
5,Jonas Jerebko,Boston Celtics,8.0,PF,29,6-10,231.0,NaN,5000000.0,Bangladesh 2.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
454,Shelvin Mack,Utah Jazz,8.0,PG,26,6-3,203.0,Butler,2433333.0,Bangladesh 2.0,NaN
455,Raul Neto,Utah Jazz,25.0,PG,24,6-1,179.0,NaN,900000.0,Bangladesh 2.0,NaN
456,Tibor Pleiss,Utah Jazz,21.0,C,26,7-3,256.0,NaN,2900000.0,Bangladesh 2.0,NaN
457,Jeff Withey,Utah Jazz,24.0,C,26,7-0,231.0,Kansas,947276.0,Bangladesh 2.0,NaN


In [ ]:
print(df.value_counts(), extra_info("rows(except rows having any Nan values) frequency.")) # for column wise counts : df.count()
print(df.sort_values(by=['College', 'Salary'], ascending=[True, False], na_position='first').to_string(), extra_info())
# For every 'Same Collage' (even Nan) the 'Salary' column sorted in Descending order. 'na_position' can't be list, only 'first' or 'last'!

Name           Team            Number  Position  Age  Height  Weight  College        Salary     Country         ct marks
Avery Bradley  Boston Celtics  0.0     PG        25   6-2     180.0   Texas          7730337.0  Bangladesh 2.0  5.0         1
Jae Crowder    Boston Celtics  99.0    SF        25   6-6     235.0   Marquette      6796117.0  Bangladesh 2.0  9.0         1
R.J. Hunter    Boston Celtics  28.0    SG        22   6-5     185.0   Georgia State  1148640.0  Bangladesh 2.0  2.0         1
Name: count, dtype: int64  --> rows(except rows having any Nan values) frequency.
------------------------------------------------------------------------------------------------------------------------

                         Name                    Team  Number Position  Age Height  Weight                College      Salary         Country  ct marks
354             Dorell Wright              Miami Heat    11.0       SF   30    6-9   205.0                    NaN         NaN  Bangladesh 2.0    

#                                                       Rank

In [ ]:
df1 = pd.DataFrame({'Name' : ['Maria', 'Aria', 'Saria', 'Akira', 'Moria'], 'Marks' : [10, 10, 20, 30, 40]})
df1.rank(ascending=[False, True]) # as we have only 2 columns, by default ascending is True for All Columns.

        # Marks    1 Indexed Position
        # -----    ------------------
        #   10            1               Since Index 1 and 2 has both 10. So their(both 10) rank = (1+2)/2 Values = 1.5
        #   10            2               
        #   20            3               20's Index = 3 and it doesn't have any duplicate value, so its rank is 3
        #   30            4
        #   40            5
        #         The Rank of 'Marks' column is sorted because the values are sorted by default. Rank of 'Name' column is not sorted
        #         because the values are not sorted but the calculation of rank is okay.

,Name,Marks
0,3.0,1.5
1,2.0,1.5
2,5.0,3.0
3,1.0,4.0
4,4.0,5.0


#                     reset_index() : Reset Current Index to its PREVIOUS INDEX `OR` Convert the Series in a DataFrame.
```js
        But What would happen if I apply reset_index() on a DataFrame or Series where I never applied set_index(..) i.e. I NEVER CHANGED THE INDEX COLUMN : In that case Pandas will think since I have to reset_index() i.e. I have to 'set the Index back to A COLUMN' and 'then set the OLD INDEX(before applying reset_index()) as the CURRENT INDEX'. So pandas will set the 'Index as a SEPERATE COLUMN. Then when it will about to set the OLD INDEX as the CURRENT INDEX, it will see I never changed the Index, so I dont have any Old Index, in that case Pandas will set the 0 BASED INDEX as the Current Index FROM ITSELF.' If A SEPERATE COLUMN added in a Series, its no longer a Series, its a DataFrame Now.
```

In [ ]:
df.head(2)

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Country,ct marks
1,Avery Bradley,Boston Celtics,0.0,PG,25,6-2,180.0,Texas,7730337.0,Bangladesh 2.0,5.0
2,Jae Crowder,Boston Celtics,99.0,SF,25,6-6,235.0,Marquette,6796117.0,Bangladesh 2.0,9.0


In [ ]:
df1 = df.set_index('Name')
print(df1.head(2), extra_info("'Name' Column is Index Column now"))
print(df1.reset_index().head(2), extra_info("The DataSet is back to its Original Form"))

# Now lets say we want to make 'Team' as the Index Column of df1.
print(df1.set_index('Team', inplace=False).head(2), extra_info("'Team' column is the Index Column now"))
# as you can see if we set 'Team' as Index, previous index 'Name' is vanished. But we want to keep 'Name' as Seperate Column 
# inside df1 and then we want to set Index = Team. First reset_index() to be the Old Main DF where 'name' is a column and then
# set_index('Team'). Now 'Name' is inside 'df1' and 'Team' is the Index.
print(df1.reset_index().set_index('Team').head(2) , extra_info("'Team' is Index now without losing 'Name' Column"))

                         Team  Number Position  Age Height  Weight    College  \
Name                                                                            
Avery Bradley  Boston Celtics     0.0       PG   25    6-2   180.0      Texas   
Jae Crowder    Boston Celtics    99.0       SF   25    6-6   235.0  Marquette   

                  Salary         Country  ct marks  
Name                                                
Avery Bradley  7730337.0  Bangladesh 2.0       5.0  
Jae Crowder    6796117.0  Bangladesh 2.0       9.0    --> 'Name' Column is Index Column now
------------------------------------------------------------------------------------------------------------------------

            Name            Team  Number Position  Age Height  Weight  \
0  Avery Bradley  Boston Celtics     0.0       PG   25    6-2   180.0   
1    Jae Crowder  Boston Celtics    99.0       SF   25    6-6   235.0   

     College     Salary         Country  ct marks  
0      Texas  7730337.0  Bangl

In [ ]:
srs = pd.Series([5, 7, 2, 3, 1], index=list('badce'))
print(srs, '    ', type(srs), extra_info("A Series"))

srsToDf = srs.reset_index()
print(srsToDf, '    ', type(srsToDf), extra_info("srs.reset_index() : 'srs' now a DataFrame"))
print(srsToDf.reset_index(), extra_info())

b    5
a    7
d    2
c    3
e    1
dtype: int64      <class 'pandas.core.series.Series'>  --> A Series
------------------------------------------------------------------------------------------------------------------------

  index  0
0     b  5
1     a  7
2     d  2
3     c  3
4     e  1      <class 'pandas.core.frame.DataFrame'>  --> srs.reset_index() : 'srs' now a DataFrame
------------------------------------------------------------------------------------------------------------------------

   level_0 index  0
0        0     b  5
1        1     a  7
2        2     d  2
3        3     c  3
4        4     e  1  --> 
------------------------------------------------------------------------------------------------------------------------



#                                                   mask(), where()
```js
        mask(cond, other, inplace):
        ---------------------------
            For 'Series':
            -------------
                Lets say based on a '1D boolean mask' we want to change a Series's value.
                cond= '1D Boolean Mask' or 'Boolean Expression that will create the 1D Boolean Mask (df3['point1'] > 7)'.
                other= For 'True' in Boolean Mask what you want to 'replace'. 'A Scaler Value' or 'A Series'|'A 1D List/ndArray'.

                       if other= 'A Series'|'A List', e.g => df3['point1'].mask(cond='1D boolean mask', other='Series'|'List/ndArray'). 
                       For every 'True' in boolean mask, the corresponding value from 'Series'|'List' will be selected to replace the old values of df3['point1'].

                       'marks' 'name' 'year'    
                df =      29     'a'   2001           Now assume we want to change the values of df['marks'] which's last digit is 9. 
                                                      obviously we need a boolean mask to tell which one we want to change.
                          32     'b'   2002
                          79     'c'   2003
                                    
                                    Hence
                                    -----
                        'marks'        'boolean mnask'
                          29                True
                          32                False
                          79                True

            
            For 'DataFrame':
            ----------------
                Lets say based on a '2D boolean mask' we want to change a DataFrame's value.
                cond= '2D Boolean Mask' or 'Boolean Expression that will create the 2D Boolean Mask (df3 > 7)'.
                other= For 'True' in Boolean Mask what you want to 'replace'. 'A Scaler Value' or 'A DataFrame'|'A 2D List/ndArray'.

                       if other= 'A DataFrame'|'A 2D List/ndArray', e.g => df3.mask(cond='2D boolean mask', other='A DataFrame'|'A 2D List/ndArray').
                       For every 'True' in boolean mask, the corresponding value from 'A DataFrame'|'A 2D List/ndArray' will be selected to replace the old values of df3.
        
        where(cond, other, inplace):
        ----------------------------
            Same as mask() but mask() replace value when its 'True' and where() replaces when its 'False'.
```

In [ ]:
print(df3, extra_info("DataFrame df3"))

print(df3['point1'].mask(cond=df3['point2'] > 7, other=df3['point2']), extra_info("mask() on a Series where other=Series"))
#           cond = False       other = 6
#                = False             = 7
#                = True   ------->   = 8
#                = True   ------->   = 9
#                = True   ------->   = 10

print(df3.mask(cond=df3 > 3, other=10), extra_info("mask() on a DataFrame where other=Scaler Value"))

print(df3.mask(cond=df3 > 3, other=df3+10), extra_info("mask() on a DataFrame where other=DataFrame"))
#           cond = False  True          other = 11     16
#                = False  True                = 12     17
#                = False  True                = 13     18
#                = True   True                = 14     19
#                = True   True                = 15     20

   point1  point2
0       1       6
1       2       7
2       3       8
3       4       9
4       5      10  --> DataFrame df3
------------------------------------------------------------------------------------------------------------------------

0     1
1     2
2     8
3     9
4    10
Name: point1, dtype: int8  --> mask() on a Series where other=Series
------------------------------------------------------------------------------------------------------------------------

   point1  point2
0       1      10
1       2      10
2       3      10
3      10      10
4      10      10  --> mask() on a DataFrame where other=Scaler Value
------------------------------------------------------------------------------------------------------------------------

   point1  point2
0       1      16
1       2      17
2       3      18
3      14      19
4      15      20  --> mask() on a DataFrame where other=DataFrame
---------------------------------------------------------------------------------

#                                                   groupby()
```js
        Look at 'groupby object.png' which is in this 'pandas' folder. After doing 'groupby(by=..)', it returns a 'DataFrameGroupBy' object. Assume 'group1' is the groubby object in that image. Each 'key' in 'group1' has its own 'value(DataFrame)'.
        Now 'group1.count()' means this 'count()' will be applied on 'each value(DataFrame)'. So doesnt matter what function you apply on (group1) except `apply()`, because that (function) will be applied on (each value(DataFrame)).

        Now after you apply functions on 'group1' you will get a 'DataFrame' result where the 'index = 'keys' of groupby object 'group1'.
        
        Usage :
        -------
                The main goal to create groupby() object is to apply the same function for each group; like sum(), mean() which are basically 'aggregate functions' except first(), last(), nth(), thats you will see that by typing'group1.' you will most functions are aggregtae functions. We can also use custom function.
        Note:
        -----
        DataFrame.apply(lambda column: ....) => DataFrame is nothing but 'bunch of columns'. When we apply() on DataFrame, 'first we grab each column MANUALLY' and 'then traverse through each value of that column manually OR can apply vectorize operation (column.is_null()..) on that column'.

        DataFrameGroupByObject.apply(lambda group: .... ) => groupby object consists of many 'group(Talking about each DatFrame, not the keys)'. When we apply() on a groupbyobject, 'first we grab each group(DataFrame)', 'then we can apply vectorize operation on the whole group(dataframe) like dataframe.count()' OR 'we can traverse each column MANUALLY like we did on DataFrame.apply(lambda column: ....)'.

        In some cases e.g. group1.nth(row_number) returns a dataframe INCLUDING THE GROUBBY COLUMN and IN ORIGINAL DATAFRAME ORDER.
        So we have to place set_index(..).sort_index() stuff to get a Sorted result, but we will barely use such fuction like nth(..) in real projects.
```

In [ ]:
names = ['Maria', 'Saria', 'Akira', 'Masha', 'Aliya', 'Alya', 'Mukail', 'Hova']
data = {                          
    'a': [5, 8, 7, 1, 6, 2, 7, 4],
    'b': [2, 1, 2, 3, 1, 3, 2, 1],
    'c': [8, 9, 0, 8, 5, 8, 3, 7],
    'd': np.random.choice(a=names, size=(8,), replace=False)}
df6 = pd.DataFrame(data)

df6_sorted = df6.sort_values(by='b')
group1_appearance = df6_sorted.drop(columns='b').set_index(keys= df6_sorted['b'].mask(cond=df6_sorted['b'].duplicated(), other='') )
print(group1_appearance, extra_info(f"groupby object {color_text('group1')}"))

group1: DataFrameGroupBy = df6.groupby(by='b')
print(group1.count(), extra_info("group1.count()"))
print(group1.sum(), extra_info("group1.sum()"))

# print(group1.mean()) # errors because mean() can't be applied on String Columns.
# print(group1[['a', 'c']]) # returns an object of group1 selecting only columns 'a' & 'c'.
print(group1[['a', 'c']].mean(), extra_info("group1[['a', 'c']].mean()"))

# Find the top 2 group which has the highest variance in 'c'. ( .index returns a list of indexes)
print( group1['c'].var().nlargest(n=2, keep='last').index , extra_info("top 2 group('b') having the highest variance in 'c'"))


#group1.get_group(name=1).set_index('b')

# applyOnGroup = group1.apply(
#         func= lambda group: group.apply(lambda column: column + ' a' if column.dtype == 'object' else column + 1),
#         include_groups= False)

# print(applyOnGroup.droplevel(1), extra_info())

   a  c       d
b              
1  8  9   Saria
   6  5  Mukail
   4  7   Maria
2  5  8    Alya
   7  3   Aliya
   7  0    Hova
3  1  8   Akira
   2  8   Masha  --> groupby object group1
------------------------------------------------------------------------------------------------------------------------

   a  c  d
b         
1  3  3  3
2  3  3  3
3  2  2  2  --> group1.count()
------------------------------------------------------------------------------------------------------------------------

    a   c                 d
b                          
1  18  21  SariaMukailMaria
2  19  11     AlyaHovaAliya
3   3  16        AkiraMasha  --> group1.sum()
------------------------------------------------------------------------------------------------------------------------

          a         c
b                    
1  6.000000  7.000000
2  6.333333  3.666667
3  1.500000  8.000000  --> group1[['a', 'c']].mean()
-------------------------------------------------------------------------

In [ ]:
print(group1_appearance, extra_info(f"groupby object {color_text('group1')}"))

# The number of groups in group1.
print(len(group1), extra_info("len(group1) = The Number of Group Numbers")) # Way 1
print(df6['b'].nunique(dropna=True), extra_info("df6['b'].nunique(dropna=True) = The Number of Group Numbers")) # Way 2

# The size(row numbers) of Each Group in group1.
print(group1.size(), extra_info("group1.size() = size(row numbers) of Each Group")) # Way 1
print(df6['b'].value_counts(sort=False, dropna=True), extra_info("df6['b'].value_counts(sort=False, dropna=True) = size(row numbers) of Each Group"))

# first = first row, last = last row, nth = 0 based nth row  OF EACH GROUP.
print(group1.first(), extra_info("group1.first() = first row of each group"))
print(group1.last(),  extra_info("group1.last()  = last row of each group"))
print(group1_appearance, extra_info(f"groupby object {color_text('group1')}"))

# group.nth(..) returns a DataFrame including the column we did groupby, in the original dataframe order. But don't worry, we won't apply nth() irl.
print(group1.nth(2).set_index('b').sort_index(), extra_info("group1.nth(2) = 3rd Row")) # group '3' is missing due to having only 2(index=0, 1) rows.
print(group1.nth([0, 2]).set_index('b').sort_index(), extra_info("group1.nth([0, 2]) = 1st & 3rd Row"))
print(group1.nth([0, -1]).set_index('b').sort_index(), extra_info("group1.nth([0, -1]) = 1st & Last Row"))

   a  c       d
b              
1  8  9   Saria
   6  5  Mukail
   4  7   Maria
2  5  8    Alya
   7  3   Aliya
   7  0    Hova
3  1  8   Akira
   2  8   Masha  --> groupby object group1
------------------------------------------------------------------------------------------------------------------------

3  --> len(group1) = The Number of Group Numbers
------------------------------------------------------------------------------------------------------------------------

3  --> df6['b'].nunique(dropna=True) = The Number of Group Numbers
------------------------------------------------------------------------------------------------------------------------

b
1    3
2    3
3    2
dtype: int64  --> group1.size() = size(row numbers) of Each Group
------------------------------------------------------------------------------------------------------------------------

b
2    3
1    3
3    2
Name: count, dtype: int64  --> df6['b'].value_counts(sort=False, dropna=True) = size(row numbers)

In [ ]:
print(group1_appearance, extra_info(f"groupby object {color_text('group1')}"))

print(group1.get_group(name=2), extra_info("group1.get_group(name=2) = returns a df including the groupby column ('b')"))
print(group1.groups, extra_info("group1.groups = {group_name = [indexes of group_name in groupby column ('b')], ....}"))
print(group1.groups.keys(), extra_info("group1.groups.keys() = groups names")) # way 1
print(df6['b'].unique(),    extra_info("             df6['b'].unique()    = groups name"))  # way 2

print(group1.describe().to_string(), extra_info("group1.describe() = describe() on each group"))
print(group1.nunique(dropna=True), extra_info("group1.nunique(dropna=True) = Number of Unique Columns on Each Group(DataFrame)"))

   a  c       d
b              
1  8  9   Saria
   6  5  Mukail
   4  7   Maria
2  5  8    Alya
   7  3   Aliya
   7  0    Hova
3  1  8   Akira
   2  8   Masha  --> groupby object group1
------------------------------------------------------------------------------------------------------------------------

   a  b  c      d
0  5  2  8   Alya
2  7  2  0   Hova
6  7  2  3  Aliya  --> group1.get_group(name=2) = returns a df including the groupby column ('b')
------------------------------------------------------------------------------------------------------------------------

{1: [1, 4, 7], 2: [0, 2, 6], 3: [3, 5]}  --> group1.groups = {group_name = [indexes of group_name in groupby column ('b')], ....}
------------------------------------------------------------------------------------------------------------------------

dict_keys([1, 2, 3])  --> group1.groups.keys() = groups names
-------------------------------------------------------------------------------------------------------

#                                                   agg() as in aggregate()
```js
        'aggregate functions' (NOT ATTRIBUTES) are those functions that return a 'SINGLE VALUE' after applying a function on a `COLUMN`; like sum(), mean(), count(), nunique(), size(), std(), first(), last(), var() .... 

        df6['d'].str.cat() => return a Single Value BUT to access it('cat()') we need to say '.str' before '.cat()'. We can access sum(), size()... after typing the column name(df6['a']) but for '.cat()' we cant without writing '.str.' first. Thats why 'cat' is not an aggregate function as pandas column can't access it DIRECTLY.
```

In [ ]:
print(group1_appearance, extra_info(f"groupby object {color_text('group1')}"))

# applying size() and nunique() function on EACH GROUP and then on EACH COLUMN INSIDE EACH GROUP.
print(group1.agg(['size', 'nunique']), extra_info("group1.agg(['size', 'nunique'])"))

# Apply different aggregate function on the columns we want.
print(group1.agg(
    {
        'a' : 'sum',
        'c' : ['nunique', 'size'],
        'd' : ['first', 'last']
    }
))

   a  c       d
b              
1  8  9   Saria
   6  5  Mukail
   4  7   Maria
2  5  8    Alya
   7  3   Aliya
   7  0    Hova
3  1  8   Akira
   2  8   Masha  --> groupby object group1
------------------------------------------------------------------------------------------------------------------------

     a            c            d        
  size nunique size nunique size nunique
b                                       
1    3       3    3       3    3       3
2    3       2    3       3    3       3
3    2       2    2       1    2       2  --> group1.agg(['size', 'nunique'])
------------------------------------------------------------------------------------------------------------------------

    a       c           d       
  sum nunique size  first   last
b                               
1  18       3    3  Saria  Maria
2  19       3    3   Alya  Aliya
3   3       1    2  Akira  Masha


In [ ]:
print(color_text("looping through groupby object :\n"))
for key, DataFrame in group1:
    print(f"key = {key}\nDataFrame :\n{DataFrame}\n----------------------")

print(color_text("\nlooping through dataframe : \n"))
for columnName in df6:
    print(columnName, end=' ')
print('\n------------------------------------------------------------------------------------------------------------------------------------------\n')

print(f"So when we traverse a {color_text('DataFrame')}, we traverse by each {color_text('column')}! But we don't go {color_text('inside')} the column, just {color_text("hold the column")}.")
print(f"That's why when we do {color_text("DataFrame.apply(lambda column: ....)")}.")
print(f"       First we Grab A Column, then either do vectorize operation(column.sum()..) or go inside that column(lambda column: lambda v: ...).")
print(f"       Then same for the next columns.\n")

print(f"And when we traverse a {color_text('Groupby object')}, we traverse by each {color_text('group(DataFrame)')} and the 'key' associated with that group(DataFrame).")
print(f"That's why when we do  {color_text("Groupby_object.apply(lambda DataFrame: lambda column: ....)")}.")
print(f"       First we Grab A group(DataFrame) and then we traverse that group(DataFrame).")
print(f"       Then same for the next groups(DataFrames).")

looping through groupby object :

key = 1
DataFrame :
   a  b  c       d
1  8  1  9   Saria
4  6  1  5  Mukail
7  4  1  7   Maria
----------------------
key = 2
DataFrame :
   a  b  c      d
0  5  2  8   Alya
2  7  2  0   Hova
6  7  2  3  Aliya
----------------------
key = 3
DataFrame :
   a  b  c      d
3  1  3  8  Akira
5  2  3  8  Masha
----------------------

looping through dataframe : 

a b c d 
------------------------------------------------------------------------------------------------------------------------------------------

So when we traverse a DataFrame, we traverse by each column! But we don't go inside the column, just hold the column.
That's why when we do DataFrame.apply(lambda column: ....).
       First we Grab A Column, then either do vectorize operation(column.sum()..) or go inside that column(lambda column: lambda v: ...).
       Then same for the next columns.

And when we traverse a Groupby object, we traverse by each group(DataFrame) and the 'key' associate

#                                                apply() on groupby object
```js
        How apply() works :

            1) 'splits' each group(DataFrame) into a Accessible DataFrame. Thats why in apply(lambda DataFrame:...), we can use the 
               DataFrame keyword as a Real DataFrame.
            2) Then do the 'operation' we set into apply(..) and get 'output(Scaler value/Series/DataFrame)' for each group(DataFrame).
            3) 'Merge' all the 'output' and return them as A SINGLE SERIES/DATAFRAME.
      
         Note: At Step 2, we must return the output we want for each group(DataFrame). Otherwise what will we MERGE at the end? YOUR MUM? NO!
               Dont think about the other groups(DataFrames), 'focus on the first group' that 'WHAT you want to return' after doing the calculation on that DataFrame. If its a Single Boolean Column, other groups will also return a Single Boolean Column from each, eventually all SEPERATED OUTPUT will be merged into a SINGLE OUTPUT. So if we understand what the first DataFrame will return, we can guess what the final merged output will be. Thats why focus on the first group(DataFrame).

                  For e.g. we want to return a DataFrame adding a new column. (speaking for the first group(DataFrame))

                  def add_new_column(dataframe):
                     dataframe['new column'] = ...... // Add The New Column
                     return dataframe                 // return the new 'dataframe' after adding the 'new column'.
                  
                  new_groupby_dataframe = groupby_object.apply(add_new_column)
                                                         --> apply(..) will also be applied on rest groups(DataFrames), ultimately will return a big big dataframe MERGING THOSE NEW DATAFRAMES.
                  
               So again : 'focus on the first group' that 'WHAT you want to return from this group(dataframe)'.
                          Ohh! All the DataFrame, Column we can access inside apply(..), they are all copy! Mess with them, change them, no issue!
```

```js
        groupby_object.apply(..) => apply() here also applied on the Column by what we did groupby(by=__). But need to ignore the column we did groupby(..) :

                groupby_object.apply(func= ...,
                                include_groups= False)
        
        groupby_object.apply(..) => returns also an 'additional index-column', hence multi-indexing which you drop by :
                result.droplevel(level=1)

```

In [ ]:
print(group1_appearance, extra_info(f"groupby object {color_text('group1')}"))

# From group1, find the names start with 'M'. group1.apply(..) returns with 'Multi-Index'.
# way 1 using apply() and boolean mask.
result =  group1.apply(
                        func= lambda dataframe: dataframe['d'] [dataframe['d'].str.startswith('M')],
                        include_groups=False                   #---------------mask----------------
                      )

# way 2 using apply()
def return_names_if_starts_with_M(v: str) -> str:
    if pd.notna(v) and v[0] == 'M':
        return v
    
result1 = group1.apply(
                        func= lambda dataframe: dataframe['d'].apply(return_names_if_starts_with_M),
                        include_groups= False
) # it also returns Nones, why? dataframe['d'].apply(..) means it will traverse EACH VALUE and return SOMETHING for EACH VALUE.
  # Since in return_names_if_starts_with_M(..) function we return v only if...., so apply(..) by default returns None if condition doesn't get match.
  # So way 1 is the only way to deal and its also cool!

result.droplevel(level=1) # drops the Multi-Index column which is at column/level 1

   a  c       d
b              
1  8  9   Saria
   6  5  Mukail
   4  7   Maria
2  5  8    Alya
   7  3   Aliya
   7  0    Hova
3  1  8   Akira
   2  8   Masha  --> groupby object group1
------------------------------------------------------------------------------------------------------------------------



b
1    Mukail
1     Maria
3     Masha
Name: d, dtype: object

In [ ]:
print(group1_appearance, extra_info(f"groupby object {color_text('group1')}"))

# Q: For each group(in group1) find the Rank based on column 'a' and create a new column('ranking on a') to set the ranking output.

def set_ranking_on_a(dataframe: pd.DataFrame):
    dataframe['ranking on a'] = dataframe['a'].rank(ascending=False)
    return dataframe
                                  
group1.apply(set_ranking_on_a, include_groups=False)
# Explanation at below block.

   a  c       d
b              
1  8  9   Saria
   6  5  Mukail
   4  7   Maria
2  5  8    Alya
   7  3   Aliya
   7  0    Hova
3  1  8   Akira
   2  8   Masha  --> groupby object group1
------------------------------------------------------------------------------------------------------------------------



a  c       d  ranking on a
b                              
1 1  8  9   Saria           1.0
  4  6  5  Mukail           2.0
  7  4  7   Maria           3.0
2 0  5  8    Alya           3.0
  2  7  0    Hova           1.5
  6  7  3   Aliya           1.5
3 3  1  8   Akira           2.0
  5  2  8   Masha           1.0

```js
        See the 'result.png' in this same 'pandas' folder.
        1) By group1.apply(..) we traverse 'group by group'|'DataFrame by DataFrame'.
        2) We picked the first DataFrame. Calculated the ranking ('dataframe['a'].rank(ascending=False)'). Set the calculated ranking 
           in a different column name 'ranking on a'. Return the New DataFrame having the column 'ranking on a'.
        3) Step 2 repeats thrice and finally we have "3 Seperated DataFrames having column 'ranking on a'". 
           Merge them into a single dataframe, the Final Output DataFrame.
```

#                                           groupby by 2 columns i.e.groupby(by=[c1, c2])
```js
            See 'sort_values and groupby.png' in this same pandas folder and you will realize both 'sort_values(by=..)' and 'groupby(by=..)' WORKS THE SAME WAY!

            groupby(by=['a', 'b']) returns a groupby object with 2 INDEX COLUMNS, inducing MULTI-INDEXING.
            Note: df7 is defined below.
```

In [ ]:
names1 = ['Maria', 'Saria', 'Akira', 'Masha', 'Aliya', 'Alya', 'Mukail', 'Hova', 'Malika']
data1 = {                          
    'a': [7, 8, 7, 1, 6, 1, 8, 7, 1],
    'b': [1, 1, 2, 3, 1, 2, 1, 2, 3],
    'c': [8, 9, 0, 8, 5, 8, 3, 7, 5],
    'd': np.random.choice(a=names1, size=(9,), replace=False)}
df7 = pd.DataFrame(data1)

group2 = df7.groupby(by=['a', 'b'])
print(group2.groups.keys(), extra_info(f"group2.groups.keys() = groups name"))
# You can even varify the answer with 'sort_values and groupby.png'.
# Now you can apply any function we used above with 'group1' object.

dict_keys([(1, 2), (1, 3), (6, 1), (7, 1), (7, 2), (8, 1)])  --> group2.groups.keys() = groups name
------------------------------------------------------------------------------------------------------------------------



#                                                       concat()
```js
        syntax : pd.concat('objs= list of dataFrames you want to concatenate', axis, ignore_index, keys)
                             --> 'We concat() only 2 dfs for teaching purposes'

        axis = 0 means concat() 'vertically' and axis = 1 means concat() 'horizontally' :
                - See 'concat(axis=1).png' and 'concat(axis=0).png' in this same 'pandas' folder.
        
        ignore_index : True of False.
                - When you concat() dataframes, their old indexes also get concatenated which can make confusion. To 'ignore concatenation' on indexes, set ignore_index=True and 0 based indexing will be applied automatically.
        
        keys : [..]
                - What if we also want to concat() on indexes i.e. keep the OLD INDEXES as it is(ignore_index=False)? If we concat() on 3 dataframes, set e.g. 'keys=['df1', 'df2', 'df3']' and you will have 'another index column' with these values which will help you to recognise which indexes belongs to which dataframe e.g. :

                'index1'  'index2'   column1   column2   column3
                  df1        0          _         _         _
                             1          _         _         _
                             2          _         _         _
                             3          _         _         _
                  df2        0          _         _         _
                             1          _         _         _
                             2          _         _         _
                             3          _         _         _
                  df3        0          _         _         _
                             1          _         _         _

                        Fig : A Multi-index DataFrame.
        
        
```

In [ ]:
df8 = pd.read_csv(r"d:\datasets\df8.csv")
df9 = pd.read_csv(r"d:\datasets\df9.csv")

df8 = df8.assign(ID  = df8['ID'].astype(np.float64),
                 Age = df8['Age'].astype(np.uint8))

print(list(df8.dtypes), extra_info("df8.dtypes"))
print(list(df9.dtypes), extra_info("df9.dtypes"))
print(df8, extra_info(color_text("df8")))
print(df9, extra_info(color_text("df9")))

print(pd.concat([df8, df9], axis=1), extra_info("pd.concat([df8, df9], axis=1)"))
print(pd.concat([df9, df8], axis=1), extra_info("pd.concat([df9, df8], axis=1)"))

print(df8, extra_info(color_text("df8")))
print(df9, extra_info(color_text("df9")))
print(pd.concat([df8, df9], axis=0), extra_info("pd.concat([df8, df9], axis=0)"))
print(pd.concat([df9, df8], axis=0), extra_info(color_text("pd.concat([df9, df8], axis=0)")))

print(pd.concat([df9, df8], axis=0, ignore_index=True), extra_info("pd.concat([df9, df8], axis=0, ignore_index=True)"))
print(pd.concat([df9, df8], axis=0, keys=['df_9', 'df_8']), extra_info("pd.concat([df9, df8], axis=0, keys=['df_9', 'df_8'])"))

[dtype('float64'), dtype('O'), dtype('uint8')]  --> df8.dtypes
------------------------------------------------------------------------------------------------------------------------

[dtype('int64'), dtype('int64')]  --> df9.dtypes
------------------------------------------------------------------------------------------------------------------------

    ID   Name  Age
0  1.0  Aaria   10
1  2.0  Baria   11
2  3.0  Caria   12
3  4.0  Daria   13
4  5.0  Earia   14
5  6.0  Faria   15
6  7.0  Garia   16
7  8.0  Haria   17  --> df8
------------------------------------------------------------------------------------------------------------------------

   Age  ID
0   20   5
1   21   8
2   22   2
3   23   3
4   24   4  --> df9
------------------------------------------------------------------------------------------------------------------------

    ID   Name  Age   Age   ID
0  1.0  Aaria   10  20.0  5.0
1  2.0  Baria   11  21.0  8.0
2  3.0  Caria   12  22.0  2.0
3  4.0  Daria   13  23.0 

#                                                       merge()
```js
        Syntax : pd.merge(left=1st DF, right=2nd DF, on='column name', how='Join Name', left_on, right_on)
             OR  left_dataframe.merge( right=2nd DF, ....................................................)

        - Watch [it](https://www.youtube.com/live/Ssy1EfK5S-o?si=46_frNd_UpzYopVo&t=2099) from [35:00].
        - Now see 'inner_join.png' and 'left_join.png' in this same 'pandas' folder.
            - 'outer' join is very rarely or doesnt used and 'right' join is ugly and confusing which can be done with left join. Thats why only 'inner' and 'left' join are drawn.

        - e.g. df10.merge(right=df11, on='id', ...) => Now the values of df11['id'] can be IN ANY ORDER i.e. the VALUES ORDER of both columns('id') in both dataframe don't need to be matched which we saw in the images.

        left_on, right_on : 
            What if The Column Name is not same? Then what is the 'on='? In that case 'left_on = Column Name from the Left(1st) DataFrame', 'right_on = Column Name from the Right(2nd) DataFrame'.
        
        how = 'right':
            pd.merge(left=df10, right=df11, on='ID', how='right') => So in that case my 2nd i.e. 'right' dataframe working as the 'first dataframe' and 'left' dataframe working as the 'second dataframe', right? Pretty confusing as we have to always look at the 'right' dataframe first. Why not just do :

                pd.merge(left=df11, right=df10, on='ID', how='left')? Because we always look at the 'left' at FIRST, then 'right'! SO WE WILL NEVER DO 'how=right'! IT's always 'inner, left, outer'.
        
        duplicate_values_on_left_dataframe :
            e.g. df10.merge(right=df11, on='id', how='inner') => Assume df10['id'] has DUPLICATES and df11['id'] doesnt. Even for duplicate values in df10['id'], we will get the corresponding values (even if its DUPLICATE) from right_dataframe, df11.
```

#                                             join()
```js
        Its same as merge(), only the name is different.

            join()  combines two dataframes on the basis of their indexes whereas the merge() is more versatile and allows us to specify 'columns' beside the index to join on for both dataframes.

            Summary : use join() on 'indices' and merge() on 'column/columns'. 
```

In [ ]:
df10 = pd.read_csv(r"d:\datasets\containing 2 dfs.csv", usecols=['ID_X', 'Course_X']) .rename(columns={'ID_X' : 'ID', 'Course_X' : 'Course'})
df11 = pd.read_csv(r"d:\datasets\containing 2 dfs.csv", usecols=['ID_Y', 'Section', 'Course_Y']) .rename(columns={'ID_Y' : 'ID', 'Course_Y' : 'Course'})

print(df10, extra_info(color_text("df10")))
print(df11, extra_info(color_text("df11")))
print(pd.merge(left=df10, right=df11, on='ID', how='inner'), extra_info(f"pd.merge(left=df10, right=df11, on='ID', how={color_text('inner')})"))
print(pd.merge(left=df10, right=df11, on='ID', how='left'), extra_info(f"pd.merge(left=df10, right=df11, on='ID', how={color_text('left')})"))

print(df10, extra_info(color_text("df10")))
print(df11, extra_info(color_text("df11")))
print(pd.merge(left=df10, right=df11, on='ID', how='right'), extra_info(f"pd.merge(left=df10, right=df11, on='ID', how={color_text('right')})"))
print(color_text("WARNING : NEVER DO how=right."), f"Do {color_text('-->')} pd.merge(left=df11, right=df10, on='ID', how='left')\n")
print(pd.merge(left=df10, right=df11, on='ID', how='outer'), extra_info(f"pd.merge(left=df10, right=df11, on='ID', how={color_text('outer')})"))

   ID Course
0   1      A
1   2      B
2   3      C
3   4      D
4   5    NaN
5   6      F
6   7    NaN  --> df10
------------------------------------------------------------------------------------------------------------------------

   ID Section Course
0  13       H      O
1  10       I      P
2   1       J      Q
3   7       K    NaN
4  11       L      R
5   3     NaN    NaN
6   5     NaN      S  --> df11
------------------------------------------------------------------------------------------------------------------------

   ID Course_x Section Course_y
0   1        A       J        Q
1   3        C     NaN      NaN
2   5      NaN     NaN        S
3   7      NaN       K      NaN  --> pd.merge(left=df10, right=df11, on='ID', how=inner)
------------------------------------------------------------------------------------------------------------------------

   ID Course_x Section Course_y
0   1        A       J        Q
1   2        B     NaN      NaN
2   3        C     NaN      N

In [ ]:
nov_reg = pd.read_csv(r"d:\datasets\CampusX\reg-november.csv")
dec_reg = pd.read_csv(r"d:\datasets\CampusX\reg-december.csv")
courses = pd.read_csv(r"d:\datasets\CampusX\courses.csv")

print(nov_reg.head(5), extra_info(color_text("nov_reg") + ".head(5)")) # 'student_id' may contain duplicate values.
print(dec_reg.head(5), extra_info(color_text("dec_reg") + ".head(5)")) # ----------------- Same -------------------
print(courses.head(5), extra_info(color_text("courses") + ".head(5)")) # 'course_id' doesn't contain any duplicate values.
# The Question is at below blocks.

   student_id  course_id
0          23          1
1          15          5
2          18          6
3          23          4
4          16          9  --> nov_reg.head(5)
------------------------------------------------------------------------------------------------------------------------

   student_id  course_id
0           3          5
1          16          7
2          12         10
3          12          1
4          14          9  --> dec_reg.head(5)
------------------------------------------------------------------------------------------------------------------------

   course_id       course_name  price
0          1            python   2499
1          2               sql   3499
2          3     data analysis   4999
3          4  machine learning   9999
4          5           tableau   2499  --> courses.head(5)
------------------------------------------------------------------------------------------------------------------------



In [ ]:
# (Assume we are selling courses)   Question 1 : How much income we did by selling our courses in both nov and dec month.
_ = ''' step 1: isin()
                            mask3 = courses['course_id'].isin(nov_reg['course_id'])
                            earned_in_nov = courses.loc[mask3, 'price'].sum()

The same way for 'nov_dec'! BUT nov_reg['course_id'] has duplicate values AND courses['course_id'] doesn't! So in mask3 we will get
only a boolean mask for EACH UNIQUE VALUE presents in nov_reg['course_id']. So isin() won't work here.

Our only option is to merge --nov_reg['course_id']-- with --courses['course_id']-- by INNER JOIN so we've corresponding 
--courses['course_id']-- values even for duplicate values in --nov_reg['course_id']--.
'''

total_reg = pd.concat(objs=[nov_reg, dec_reg], axis=0)

print( total_reg.merge(right=courses, on='course_id', how='inner') ['price'].sum() )
#      ----------------------------------------------------------- ---------------

154247


In [ ]:
#                          Question 2 : How much income we did by selling our courses in both nov and dec month SEPARATELY.

nov_reg_and_courses = nov_reg.merge(right=courses, on='course_id', how='inner') ['price'].sum()
dec_reg_and_courses = dec_reg.merge(right=courses, on='course_id', how='inner') ['price'].sum()

print(f"Total Income for November Month = {nov_reg_and_courses}")
print(f"Total Income for December Month = {dec_reg_and_courses}\n")

print(color_text("Another Way : (CampusX did)\n"))

total_reg = pd.concat(objs=[nov_reg, dec_reg], axis=0, keys=['Nov', 'Dec']) .reset_index() # Multi-indexing became FIRST TWO COLUMNS after reset_index()
total_reg_and_courses = total_reg.merge(right=courses, on='course_id', how='inner') # See the answer in the below block.

print( total_reg_and_courses.groupby(by='level_0') ['price'].sum() ) # The group_by key's values are sorted by default(Dec, Nov)

Total Income for November Month = 89175
Total Income for December Month = 65072

Another Way : (CampusX did)

level_0
Dec    65072
Nov    89175
Name: price, dtype: int64


In [ ]:
print(color_text("    total_reg.merge(right=courses, on='course_id', how='inner')\n"))
print(total_reg_and_courses)

    total_reg.merge(right=courses, on='course_id', how='inner')

   level_0  level_1  student_id  course_id       course_name  price
0      Nov        0          23          1            python   2499
1      Nov        1          15          5           tableau   2499
2      Nov        2          18          6          power bi   1899
3      Nov        3          23          4  machine learning   9999
4      Nov        4          16          9            plotly    699
5      Nov        5          18          1            python   2499
6      Nov        6           1          1            python   2499
7      Nov        7           7          8            pandas   1099
8      Nov        8          22          3     data analysis   4999
9      Nov        9          15          1            python   2499
10     Nov       10          19          4  machine learning   9999
11     Nov       11           1          6          power bi   1899
12     Nov       12           7         10         

In [ ]:
students = pd.read_csv(r"d:\Datasets\CampusX\students.csv")
print(students, extra_info(color_text("students")))
print(nov_reg.head(5), extra_info(color_text("nov_reg") + ".head(5)")) # 'student_id' may contain duplicate values.
print(dec_reg.head(5), extra_info(color_text("dec_reg") + ".head(5)")) # ----------------- Same -------------------
print(courses.head(5), extra_info(color_text("courses") + ".head(5)")) # 'course_id' doesn't contain any duplicate values.
# The Q is in the below block.

    student_id                name  partner
0            1       Kailash Harjo       23
1            2         Esha Butala        1
2            3      Parveen Bhalla        3
3            4         Marlo Dugal       14
4            5         Kusum Bahri        6
5            6  Lakshmi Contractor       10
6            7        Tarun Thaker        9
7            8      Radheshyam Dey        5
8            9   Nitika Chatterjee        4
9           10      Aayushman Sant        8
10          11  David Mukhopadhyay       20
11          12          Radha Dutt       19
12          13      Munni Varghese       24
13          14    Pranab Natarajan       22
14          15           Preet Sha       16
15          16        Elias Dodiya       25
16          17        Yasmin Palan        7
17          18     Fardeen Mahabir       13
18          19        Qabeel Raman       12
19          20       Hanuman Hegde       11
20          21          Seema Kota       15
21          22          Yash Set

In [ ]:
#       Question 3 : Find those students info who enrolled in our courses in BOTH NOVEMBER & DECEMBER.

# pd.concat([nov_reg['student_id'], dec_reg['student_id']], axis=0)  # One way to intersect using PANDAS ONLY.

all_reg_students_id = np.intersect1d(nov_reg['student_id'], dec_reg['student_id'])
students[ mask3 := students['student_id'].isin(all_reg_students_id) ]

,student_id,name,partner
0,1,Kailash Harjo,23
2,3,Parveen Bhalla,3
6,7,Tarun Thaker,9
10,11,David Mukhopadhyay,20
15,16,Elias Dodiya,25
16,17,Yasmin Palan,7
17,18,Fardeen Mahabir,13
21,22,Yash Sethi,21
22,23,Chhavi Lachman,18


In [ ]:
#                               Question 4 :  From 'students' table Find Partner Name for each Student.
 
_ = '''             students                                                       students
    student_id        name       "partner"                         "student_id"      name        partner
        1       Kailash Harjo       23                                  1       Kailash Harjo       23
        2         Esha Butala        1                                  2         Esha Butala        1
        3      Parveen Bhalla        3                                  3      Parveen Bhalla        3
   .......................................                        .......................................
        23     Chhavi Lachman       18                                  23     Chhavi Lachman       18
'''
# 'partner's ID is the student_id itself. See 'inner_join.png' in this 'pandas' folder if can't think the answer. 

temp_df = students.merge(right=students, left_on= 'partner', right_on= 'student_id', how='inner') # 'partner' 100% exists in 'student_id'. So 'inner' or 'left' doesn't matter.
print(temp_df.head(5).to_string(), extra_info("temp_df.head(5)"))

temp_df[['name_x', 'name_y']] .rename(columns={'name_x': 'Student Name', 'name_y': 'Partner Name'})

   student_id_x          name_x  partner_x  student_id_y              name_y  partner_y
0             1   Kailash Harjo         23            23      Chhavi Lachman         18
1             2     Esha Butala          1             1       Kailash Harjo         23
2             3  Parveen Bhalla          3             3      Parveen Bhalla          3
3             4     Marlo Dugal         14            14    Pranab Natarajan         22
4             5     Kusum Bahri          6             6  Lakshmi Contractor         10  --> temp_df.head(5)
------------------------------------------------------------------------------------------------------------------------



,Student Name,Partner Name
0,Kailash Harjo,Chhavi Lachman
1,Esha Butala,Kailash Harjo
2,Parveen Bhalla,Parveen Bhalla
3,Marlo Dugal,Pranab Natarajan
4,Kusum Bahri,Lakshmi Contractor
5,Lakshmi Contractor,Aayushman Sant
6,Tarun Thaker,Nitika Chatterjee
7,Radheshyam Dey,Kusum Bahri
8,Nitika Chatterjee,Marlo Dugal
9,Aayushman Sant,Radheshyam Dey


In [ ]:
print(students.head(5), extra_info(color_text("students") + ".head(5)"))
print(nov_reg.head(5), extra_info(color_text("nov_reg") + ".head(5)")) # 'student_id' may contain duplicate values.
print(dec_reg.head(5), extra_info(color_text("dec_reg") + ".head(5)")) # ----------------- Same -------------------
print(courses.head(5), extra_info(color_text("courses") + ".head(5)")) # 'course_id' doesn't contain any duplicate values.
# The Q in the below block.

   student_id            name  partner
0           1   Kailash Harjo       23
1           2     Esha Butala        1
2           3  Parveen Bhalla        3
3           4     Marlo Dugal       14
4           5     Kusum Bahri        6  --> students.head(5)
------------------------------------------------------------------------------------------------------------------------

   student_id  course_id
0          23          1
1          15          5
2          18          6
3          23          4
4          16          9  --> nov_reg.head(5)
------------------------------------------------------------------------------------------------------------------------

   student_id  course_id
0           3          5
1          16          7
2          12         10
3          12          1
4          14          9  --> dec_reg.head(5)
------------------------------------------------------------------------------------------------------------------------

   course_id       course_name  pric

In [ ]:
#                           Question 5 : Find TOP 3 Students who has the MOST ENROLLMENT NUMBERS.

total_reg = pd.concat([nov_reg, dec_reg], axis=0)
top_3_student_ID = total_reg.value_counts(subset='student_id', dropna=True, sort=False).nlargest(n=3)

students[ mask4 := students['student_id'].isin(top_3_student_ID.index) ]

,student_id,name,partner
0,1,Kailash Harjo,23
6,7,Tarun Thaker,9
22,23,Chhavi Lachman,18


In [ ]:
#                           Question 6 : Top 3 Students who spent most money in buying courses.
start = time()

print( pd.concat([nov_reg, dec_reg], axis=0). merge(students, on='student_id', how='inner') .merge(courses, on='course_id', how='inner')
        .groupby(['student_id', 'name']) ['price'].sum().nlargest(3) ) # multi-indexing by groupby-ing 2 columns.

print(color_text(f"{time() - start}"))

print(color_text("\n              Second Way : \n")) # -------------------------------------------------------------------------------

start = time()
top_3_student_ID = ( pd.concat([nov_reg, dec_reg], axis=0) .merge(courses, on='course_id', how='inner')
                     .groupby('student_id')['price'].sum().nlargest(3) .reset_index() )

print( top_3_student_ID.merge(students[['student_id', 'name']], on='student_id', how='inner') )
print(color_text(f"{time() - start}"))

_ = '''
        The Reason First Method taking much time because We first concat(), merge(), merge() and now we have very big data to deal 
        with.

        In the Second method we concat(), merge() AND then :
            top_3_student_ID.merge(students[['student_id', 'name']], on='student_id', how='inner') :
                Here we merged but the LEFT TABLE = A DataFrame with ONLY 3 ROWS and we also STRICTLY said :
                    right= students[['student_id', 'name']] :
                        so that after merge() we don't have to again select the columns we want to select.

'''

student_id  name            
23          Chhavi Lachman      22594
14          Pranab Natarajan    15096
19          Qabeel Raman        13498
Name: price, dtype: int64
0.0360414981842041

              Second Way : 

   student_id  price              name
0          23  22594    Chhavi Lachman
1          14  15096  Pranab Natarajan
2          19  13498      Qabeel Raman
0.004986286163330078
